![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [10]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/features.txt"
filename_labels = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "/Users/rayssaathayde/Library/CloudStorage/OneDrive-Pessoal/Ciências de Dados/EBAC/27/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze('columns')
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze('columns')
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze('columns')
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [11]:
#separar base para validação

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=100)

In [22]:
%%time

clf = DecisionTreeClassifier(ccp_alpha= 0.001, random_state=100)

clf = clf.fit(X_train, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {round(clf.score(X_train, y_train), 2)}')

# Avaliação da acurácia do classificador nos dados de validação
print(f'Acurácia na base de validação: {round(clf.score(X_valid, y_valid), 2)}')

# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {round(clf.score(X_test, y_test), 2)}')

Acurácia na base de treinamento: 0.98
Acurácia na base de validação: 0.94
Acurácia na base de teste: 0.87
CPU times: user 2.4 s, sys: 17.4 ms, total: 2.41 s
Wall time: 2.44 s


## Árvore com PCA

Faça uma análise de componentes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [25]:
%%time

# Aplica o PCA com 1 componente aos dados de treinamento
prcomp = PCA(n_components=1).fit(X_train)

# Transforma os dados de treinamento, validação e teste utilizando as componentes principais encontradas pelo PCA
pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)

# Cria árvore de decisão com ccp_alpha=0.001 e treina-a com os dados de treinamento transformados
clf = DecisionTreeClassifier(ccp_alpha= 0.001, random_state=100)

clf = clf.fit(pc_treino, y_train)

# Avaliação da acurácia do classificador nos dados de treinamento
print(f'Acurácia na base de treinamento: {round(clf.score(pc_treino, y_train), 2)}')

# Avaliação da acurácia do classificador nos dados de validação
print(f'Acurácia na base de validação: {round(clf.score(pc_valida, y_valid), 2)}')

# Avaliação da acurácia do classificador nos dados de teste
print(f'Acurácia na base de teste: {round(clf.score(pc_teste, y_test), 2)}')

Acurácia na base de treinamento: 0.5
Acurácia na base de validação: 0.49
Acurácia na base de teste: 0.46
CPU times: user 846 ms, sys: 191 ms, total: 1.04 s
Wall time: 159 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [32]:
%%time

componentes = [1, 2, 5, 10, 50, 100]

for n in componentes:
    # Aplica o PCA com n componentes aos dados de treinamento
    prcomp = PCA(n_components=n).fit(X_train)
    
    # Transforma os dados de treinamento, validação e teste utilizando as componentes principais encontradas pelo PCA
    pc_treino = prcomp.transform(X_train)
    pc_valida = prcomp.transform(X_valid)
    pc_teste  = prcomp.transform(X_test)
    
    # Cria árvore de decisão com ccp_alpha=0.001 e treina-a com os dados de treinamento transformados
    clf = DecisionTreeClassifier(ccp_alpha= 0.001, random_state=100)
    clf = clf.fit(pc_treino, y_train)

    # Avaliação da acurácia do classificador nos dados de treinamento
    print(f'Acurácia na base de treinamento: {round(clf.score(pc_treino, y_train), 2)} com {n} componentes.')
    
    # Avaliação da acurácia do classificador nos dados de validação 
    print(f'Acurácia na base de validação: {round(clf.score(pc_valida, y_valid), 2)} com {n} componentes.')
    
    # Avaliação da acurácia do classificador nos dados de teste
    print(f'Acurácia na base de teste: {round(clf.score(pc_teste, y_test), 2)} com {n} componentes.\n')

Acurácia na base de treinamento: 0.5 com 1 componentes.
Acurácia na base de validação: 0.49 com 1 componentes.
Acurácia na base de teste: 0.46 com 1 componentes.

Acurácia na base de treinamento: 0.62 com 2 componentes.
Acurácia na base de validação: 0.6 com 2 componentes.
Acurácia na base de teste: 0.58 com 2 componentes.

Acurácia na base de treinamento: 0.85 com 5 componentes.
Acurácia na base de validação: 0.82 com 5 componentes.
Acurácia na base de teste: 0.8 com 5 componentes.

Acurácia na base de treinamento: 0.9 com 10 componentes.
Acurácia na base de validação: 0.86 com 10 componentes.
Acurácia na base de teste: 0.83 com 10 componentes.

Acurácia na base de treinamento: 0.93 com 50 componentes.
Acurácia na base de validação: 0.87 com 50 componentes.
Acurácia na base de teste: 0.81 com 50 componentes.

Acurácia na base de treinamento: 0.94 com 100 componentes.
Acurácia na base de validação: 0.86 com 100 componentes.
Acurácia na base de teste: 0.81 com 100 componentes.

CPU time

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

- Ao reduzir a dimensionalidade utilizando a técnica de Análise de Componentes Principais (PCA), observamos que a principal mudança e mais perceptível ocorre em relação ao tempo de processamento. A árvore de decisão executada com o conjunto de dados completo, contendo todas as 561 variáveis, levou aproximadamente 2,40 segundos para ser executada, resultando em uma acurácia de 87% na base de teste e 94% na base de validação. Ao utilizar o PCA com apenas um componente, o tempo de processamento foi reduzido para pouco menos de 160 ms, porém a acurácia na base de teste foi de 46% e na de validação foi 49%.

- Aumentando o número de componentes, foi possível aumentar gradualmente a acurácia nas bases de teste e validação. Com 50 componentes, foi atingida uma acurácia de até 81% na base de teste e 86% na de validação, com um tempo de processamento de quae 3 segundos para executar todas as 6 combinações de números diferentes de componentes. Os resultados com 100 componentes não foram muito diferentes do que com 50 componentes. 
- Portanto, podemos concluir que é possível obter uma acurácia próxima à árvore com todas as variáveis do conjunto de dados, porém com um tempo de processamento significativamente reduzido. 